# UCI Heart Disease Detection

This notebook is used as part of my thesis on XAI, comparing different XAI methods and libraries.
<br/>
The purpose of the created models is to classify if a patient is either healthy or has a heart disease.
<br/>
<br/>
Attributes:
1. age
2. sex (1=male, 0=female)
3. chest pain type (4 values)
4. resting blood pressure
5. serum cholesterol in milligrams per deciliter (mg/dl)
6. fasting blood sugar > 120 mg/dl
7. resting electrocardiographic results (values 0,1,2)
8. maximum heart rate achieved
9. exercise induced angina
10. oldpeak = ST depression induced by exercise relative to rest
11. the slope of the peak exercise ST segment
12. number of major vessels (0-3) colored by flourosopy
13. thal: 3 = normal; 6 = fixed defect; 7 = reversable defect

Dataset: https://archive.ics.uci.edu/ml/datasets/Heart+Disease

## 1 Set up Environment and Dataset <a class="anchor" id="ch1"></a>

### 1.1 Load Libraries and Set Up Parameters <a class="anchor" id="ch1.1"></a>

In [ ]:
# random seed for reproduction
seedNum = 23

In [ ]:
# import dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import urllib.request
import seaborn as sns
import catboost
import shap
import lime
import graphviz
import tensorflow as tf

from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_validate
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, auc
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.inspection import partial_dependence, plot_partial_dependence

from catboost import CatBoostClassifier
from alibi.explainers import AnchorTabular, CounterFactualProto, CounterFactual
from alibi.utils.mapping import ohe_to_ord, ord_to_ohe
from datetime import datetime

# required installs:
# pip install shap
# pip install lime
# pip install alibi
# conda install python-graphviz AND install from https://graphviz.org/download/

In [ ]:
# timer for the script processing
startTimeScript = datetime.now()

# set up n_jobs
n_jobs = 6

# set flag for splitting the dataset
splitDataset = True
splitPercentage = 0.20

# set number of folds for cross validation
n_folds = 10

# set various default modeling parameters
scoring = 'accuracy'

1. age
2. sex
3. chest pain type (4 values)
4. resting blood pressure
5. serum cholesterol in milligrams per deciliter (mg/dl)
6. fasting blood sugar > 120 mg/dl
7. resting electrocardiographic results (values 0,1,2)
8. maximum heart rate achieved
9. exercise induced angina
10. oldpeak = ST depression induced by exercise relative to rest
11. the slope of the peak exercise ST segment
12. number of major vessels (0-3) colored by flourosopy
13. thal: 3 = normal; 6 = fixed defect; 7 = reversable defect

In [ ]:
# order of columns for explanation compatability
new_order=["sex","cp","fbs","restecg","exang","slope","thal","age","trestbps","chol","thalach","oldpeak","ca","target"]

# dictionary of categorical variable values
category_map={0: ["female", "male"],
              1: ["typical angina","atypical angina","non-anginal pain","asymptomatic"],
              2: ["below 120 mg/dl","above 120 ml/dl"],
              3: ["normal","ST-T wave abnormality","probable left ventricular hypertrophy"],
              4: ["no","yes"],
              5: ["upsloping","flat","downsloping"],
              6: ["no info","normal","fixed defect","reversable defect"]
             }

# dict of column names for renaming
col_names = {"cp":'chest pain type', "trestbps":'resting blood pressure', "chol":'serum cholesterol (mg/dl)',
             "fbs":'fasting blood sugar', "restecg":'resting ecg results',
             "thalach":'maximum heart rate achieved', "exang":'exercise induced angina',
             "oldpeak":'exercise induced ST depression',
             "slope":'slope of peak exercise ST segment', "ca":'vessels colored by flourosopy', "thal":"thalassemia type"}

In [ ]:
#import dataset
dataset_path = 'data/heart.csv'
Xy_original = pd.read_csv(dataset_path)
Xy_original = Xy_original[new_order]
Xy_original.rename(columns=col_names, inplace=True)
Xy_original.shape

### 1.2 Preprocessing <a class="anchor" id="ch1.2"></a>

In [ ]:
# Use variable totCol to hold the number of columns in the dataframe
totCol = len(Xy_original.columns)
totAttr = totCol-1

X_original = Xy_original.iloc[:,0:totAttr]
y_original = Xy_original.iloc[:,totAttr]

print("Xy_original.shape: {} X_original.shape: {} y_original.shape: {}".format(Xy_original.shape, X_original.shape, y_original.shape))

In [ ]:
# create dictionary with the number of categories for each variable in the dataset
cat_vars_ord = {}
n_categories = len(list(category_map.keys()))
for i in range(n_categories):
    cat_vars_ord[i] = len(np.unique(X_original.to_numpy()[:, i]))
print(cat_vars_ord)

In [ ]:
cat_vars_ohe = ord_to_ohe(X_original.to_numpy(), cat_vars_ord)[1]
print(cat_vars_ohe)

In [ ]:
X_num = X_original.to_numpy()[:, -6:].astype(np.float32, copy=False)
scaler = MinMaxScaler(feature_range=(-1,1))
X_num_scaled= scaler.fit_transform(X_num)

In [ ]:
X_cat = X_original.to_numpy()[:, :-6].copy()
ohe = OneHotEncoder(categories='auto')
ohe.fit(X_cat)
X_cat_ohe = ohe.transform(X_cat)

In [ ]:
X_enc = np.c_[X_cat_ohe.todense(), X_num_scaled].astype(np.float32, copy=False)

X_enc = pd.DataFrame(X_enc)

In [ ]:
# Split the data further into training and test datasets
X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(X_enc, y_original, test_size=splitPercentage, 
                                                                stratify=y_original, random_state=seedNum)

print("X_train.shape: {} y_train_df.shape: {}".format(X_train_df.shape, y_train_df.shape))
print("X_test_df.shape: {} y_test_df.shape: {}".format(X_test_df.shape, y_test_df.shape))

In [ ]:
# Finalize the training and testing datasets for the modeling activities
X_train = X_train_df.to_numpy()
y_train = y_train_df.to_numpy()
X_test = X_test_df.to_numpy()
y_test = y_test_df.to_numpy()
print("X_train.shape: {} y_train.shape: {}".format(X_train.shape, y_train.shape))
print("X_test.shape: {} y_test.shape: {}".format(X_test.shape, y_test.shape))

## 2 Tree-based Modeling <a class="anchor" id="ch2"></a>

Random Forest:

In [ ]:
startTimeModule = datetime.now()

tune_model = RandomForestClassifier(random_state=seedNum, n_jobs=n_jobs)

n_estimators = [100]
criterion = ["gini","entropy"]
max_features =[None, "sqrt", 0.2, 0.3, 0.4, 0.5]

paramGrid = dict(n_estimators=n_estimators, criterion=criterion, max_features=max_features)

kfold = KFold(n_splits=n_folds)
grid = GridSearchCV(estimator=tune_model, param_grid=paramGrid, scoring=scoring, cv=kfold, refit="Recall")
grid_result = grid.fit(X_enc, y_original)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
print ('Computing time:',(datetime.now() - startTimeModule))

clf_rf_be = grid_result.best_estimator_
clf_rf = clf_rf_be.fit(X_train, y_train)

Gradient Boosting:

In [ ]:
clf_cb_be = CatBoostClassifier(eval_metric='Accuracy', depth=6, verbose=False)
clf_cb = clf_cb_be.fit(X_train, y_train, verbose=False)

Evaluation:

In [ ]:
predictions_rf = clf_rf.predict(X_test)
predictions_cb = clf_cb.predict(X_test)
cv_rf = cross_val_score(clf_rf_be, X_train, y_train, cv=kfold, scoring=scoring)
cv_cb = cross_val_score(clf_cb_be, X_train, y_train, cv=kfold, scoring=scoring)

print(clf_rf,"\nConfusion Matrix:")
print(confusion_matrix(y_test, predictions_rf))
print("\n\nClassification Report:\n\n",classification_report(y_test, predictions_rf))
print("Cross-Validation: %f (%f)" % (cv_rf.mean(), cv_rf.std()))
print("--------------------------------------------------------\n")

print(clf_cb,"\nConfusion Matrix:")
print(confusion_matrix(y_test, predictions_cb))
print("\n\nClassification Report:\n\n",classification_report(y_test, predictions_cb))
print("Cross-Validation: %f (%f)" % (cv_cb.mean(), cv_cb.std()))

## 3 Counterfactuals <a class="anchor" id="ch3"></a>

In [ ]:
clf=clf_cb
pred_idx = 60

probabilities = clf.predict_proba(X_test)
print("Probabilities: ", probabilities[pred_idx])
print("Correct class: ", y_test[pred_idx])

In [ ]:
target_names = ["healthy", "sick"]
feature_names = X_original.columns.values

In [ ]:
x = X_test[pred_idx].reshape((1,) + X_test[0].shape)

In [ ]:
predict_fn = lambda x: clf.predict_proba(x)

In [ ]:
x = X_test[pred_idx].reshape((1,) + X_test[0].shape)

shape = x.shape
beta = .01
c_init = 1.
c_steps = 5
max_iterations = 500
rng = (-1., 1.)  # scale features between -1 and 1
rng_shape = (1,) + X_original.shape[1:]
feature_range = ((np.ones(rng_shape) * rng[0]).astype(np.float32),
                 (np.ones(rng_shape) * rng[1]).astype(np.float32))
use_kdtree = True
theta = 10.  # weight of prototype loss term



tf.compat.v1.disable_eager_execution()

cf = CounterFactualProto(predict_fn,
                         shape,
                         beta=beta,
                         theta=theta,
                         cat_vars=cat_vars_ohe,
                         ohe=True,
                         use_kdtree=use_kdtree,
                         max_iterations=max_iterations,
                         feature_range=feature_range,
                         c_init=c_init,
                         c_steps=c_steps,
                         eps=(0.05, 0.05)
                        )

cf.fit(X_train, d_type='abdm');

In [ ]:
startTimeModule = datetime.now()
explanation = cf.explain(x)
print ('Computing time:',(datetime.now() - startTimeModule))

In [ ]:
def describe_instance(X, explanation, eps=1e-2):
    print('Prediction by the model: {}  -- proba: {}'.format(target_names[explanation.orig_class],
                                                       explanation.orig_proba[0]))
    print('Counterfactual instance: {}  -- proba: {}'.format(target_names[explanation.cf['class']],
                                                             explanation.cf['proba'][0]))
    print('\nCounterfactual perturbations...')
    
    print('\nCategorical:')
    X_orig_ord = ohe_to_ord(X, cat_vars_ohe)[0]
    X_cf_ord = ohe_to_ord(explanation.cf['X'], cat_vars_ohe)[0]
    delta_cat = {}
    for i, (_, v) in enumerate(category_map.items()):
        cat_orig = v[int(X_orig_ord[0, i])]
        cat_cf = v[int(X_cf_ord[0, i])]
        if cat_orig != cat_cf:
            delta_cat[feature_names[i]] = [cat_orig, cat_cf]
    if delta_cat:
        for k, v in delta_cat.items():
            print('{}: {}  -->   {}'.format(k, v[0], v[1]))
    
    print('\nNumerical:')
    delta_num = X_cf_ord[0, -6:] - X_orig_ord[0, -6:]
    n_keys = len(list(cat_vars_ord.keys()))
    X_orig_num = scaler.inverse_transform(X_orig_ord[0,-6:].reshape(1,-1))
    X_cf_num = scaler.inverse_transform(X_cf_ord[0,-6:].reshape(1,-1))
    for i in range(delta_num.shape[0]):
        if np.abs(delta_num[i]) > eps:
            print('{}: {:.2f}  -->   {:.2f}'.format(feature_names[i+n_keys],
                                            #X_orig_ord[0,i+n_keys],
                                            X_orig_num[0,i],
                                            #X_cf_ord[0,i+n_keys]))
                                            X_cf_num[0,i]))      

In [ ]:
def describe_instance2(X, explanation, eps=1e-2):
    print('Nearest counterfactual instance: {}'.format(target_names[explanation.cf['class']]))
    print('Probabilities: ',round(explanation.cf['proba'][0][0],2)," ",round(explanation.cf['proba'][0][1],2))
    
    print('\nSmallest feature value changes necessary:\n')
    
    #print('\nCategorical:')
    X_orig_ord = ohe_to_ord(X, cat_vars_ohe)[0]
    X_cf_ord = ohe_to_ord(explanation.cf['X'], cat_vars_ohe)[0]
    delta_cat = {}
    for i, (_, v) in enumerate(category_map.items()):
        cat_orig = v[int(X_orig_ord[0, i])]
        cat_cf = v[int(X_cf_ord[0, i])]
        if cat_orig != cat_cf:
            delta_cat[feature_names[i]] = [cat_orig, cat_cf]
    if delta_cat:
        for k, v in delta_cat.items():
            print('{}: {}  -->   {}'.format(k, v[0], v[1]))
    
    #print('\nNumerical:')
    delta_num = X_cf_ord[0, -6:] - X_orig_ord[0, -6:]
    n_keys = len(list(cat_vars_ord.keys()))
    X_orig_num = scaler.inverse_transform(X_orig_ord[0,-6:].reshape(1,-1))
    X_cf_num = scaler.inverse_transform(X_cf_ord[0,-6:].reshape(1,-1))
    for i in range(delta_num.shape[0]):
        if np.abs(delta_num[i]) > eps:
            print('{}: {:.2f}  -->   {:.2f}'.format(feature_names[i+n_keys],
                                            #X_orig_ord[0,i+n_keys],
                                            X_orig_num[0,i],
                                            #X_cf_ord[0,i+n_keys]))
                                            X_cf_num[0,i]))      

In [ ]:
describe_instance2(x, explanation)